# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'heartfailure-dataset'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RMLYUR6Z8 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_DS3_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "compute-instance-ds3-v2"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2', max_nodes=4)

    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [37]:
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.core.environment import Environment

import os

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": uniform(0.05, 1),
    "--max_iter": choice(20, 40, 60, 80, 100, 1000)
    }
)

script_folder = "."

#TODO: Create your estimator and hyperdrive config
sklearn_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")
run_config = ScriptRunConfig(source_directory=script_folder, compute_target=cpu_cluster, script="train.py", environment=sklearn_environment)

if "outputs" not in os.listdir():
    os.mkdir("./outputs")

hyperdrive_run_config = HyperDriveConfig(run_config=run_config, policy=early_termination_policy, hyperparameter_sampling=param_sampling, 
    primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=50, max_concurrent_runs=2)

In [38]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config, show_output=True)

In [39]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c5cd0fee-ff03-4214-8398-1c11157c948f
Web View: https://ml.azure.com/experiments/heartfailure-dataset/runs/HD_c5cd0fee-ff03-4214-8398-1c11157c948f?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-137811/workspaces/quick-starts-ws-137811

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-07T12:48:49.301129][API][INFO]Experiment created<END>\n""<START>[2021-02-07T12:48:50.183548][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2021-02-07T12:48:49.894281][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n"<START>[2021-02-07T12:48:50.8478449Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service